<a href="https://colab.research.google.com/github/baptiste-bedouret/Mistral7B-Finetuned/blob/master/Fine-tuning%20Mistral%207B%20on%20annotated%20dataset%20V2..ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup Runtime
For fine-tuning Mistral, a GPU instance is essential. Follow the directions below:

1. Go to `Runtime` (located in the top menu bar).
2. Select `Change Runtime Type`.
3. Choose `T4 GPU` (or a comparable option).


## Packages installation

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install transformers accelerate trl torch bitsandbytes peft datasets -qU

## Load the dataset

In [ ]:
from datasets import load_dataset, DatasetDict, Dataset
from sklearn.model_selection import train_test_split

dataset = (load_dataset("json", data_files="/content/drive/My Drive/Smart-Data/Renault/Dataset_Annotated.json",
                        split='train').train_test_split(train_size=3500, test_size=1000))
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['tasks', 'completions'],
        num_rows: 3500
    })
    test: Dataset({
        features: ['tasks', 'completions'],
        num_rows: 1000
    })
})


Remove the rows when the language is different than english:

In [ ]:
# def filter_dataset_by_language(dataset_dict, target_language="EN"):
#     filtered_data = []
#     for example in dataset_dict["train"]:
#         if example["tasks"]["langue"] == target_language:
#             filtered_data.append(example)

#     return Dataset.from_dict({"train": filtered_data})

# dataset = filter_dataset_by_language(dataset)
# print(dataset)

Remove columns from 'tasks':

In [ ]:
columns_to_remove = ['id', 'date', 'pays', 'langue', 'score']

def remove_columns_from_tasks(entry):
    for column in columns_to_remove:
        entry['tasks'].pop(column, None)
    return entry

# Apply the function to each entry in the dataset
dataset = dataset.map(remove_columns_from_tasks)

Map:   0%|          | 0/3500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
print(dataset['train']['tasks'][2]['text'])

Very happy about the service and the MOT but I was expecting my car to be cleaned up as usual and it wasn't! Little bit disappointed with that but the team was lovely.


In [ ]:
print(dataset['test']['tasks'][21]['text'])

Doing Well - Very attendive initial approach.
To be Improved - Followup/update to the customer can be in the same way.


Remove columns from 'completions':

In [ ]:
completions_columns_to_remove = ['intensity', 'span']

def remove_columns_from_completions(entry):
    for completion in entry['completions']:
        for column in completions_columns_to_remove:
            completion.pop(column, None)
    return entry

# Apply the function to each entry in the dataset
dataset = dataset.map(remove_columns_from_completions)

Map:   0%|          | 0/3500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
print(dataset['train']['completions'][2])

[{'category': 'Cleanliness-State of vehicle', 'polarity': 'negative'}, {'category': 'Welcome-Kindness-Warmth-Friendliness', 'polarity': 'positive'}]


In [ ]:
print(dataset['test']['completions'][21])

[{'category': 'Listenning-Care', 'polarity': 'positive'}, {'category': 'Followup/update to the customer', 'polarity': 'negative'}]


Load the dataset containing the categories:

In [ ]:
category_list = load_dataset("csv", data_files="/content/drive/My Drive/Smart-Data/Renault/Dataset_categories.csv")
print(category_list)

DatasetDict({
    train: Dataset({
        features: ['CONCEPTS NIVEAU 2'],
        num_rows: 72
    })
})


In [ ]:
print(category_list['train']['CONCEPTS NIVEAU 2'][:5])

['Welcome-Kindness-Warmth-Friendliness', 'Listenning-Care', 'Attention-Assistance-Effort', 'Correct contact', 'Quality of the relationship']


Create formated prompt:

```
<s>[INST]### Instruction:
Classify this text in one or more of the following categories:
{list of categories}
[/INST]

[INST]### Additional Instruction:
For each of these categories, indicate whether the text is positive, neutral, or negative.
[/INST]

### Input:
{input}

### Response:
{response}</s>
```

In [ ]:
def formatting_prompts_func(example, category_examples):
    output_texts = []
    for i in range(len(example['completions'])):
        text = f"""
 <s>[INST]### Instruction:
 Classify this text in one or more of the following categories:
 {category_examples['train']['CONCEPTS NIVEAU 2']}
 [/INST]

 [INST]### Additional Instruction:
 For each of these categories, indicate whether the text is positive, neutral, or negative.
 [/INST]

 ### Input:\n{example["tasks"][i]['text']}

 ### Response:\n{example["completions"][i]}</s>"""
        output_texts.append(text)
    return output_texts

In [ ]:
print(formatting_prompts_func(dataset['train'], category_list))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
print(formatting_prompts_func(dataset['train'][:5], category_list))

["\n <s>[INST]### Instruction:\n Classify this text in one or more of the following categories:\n ['Welcome-Kindness-Warmth-Friendliness', 'Listenning-Care', 'Attention-Assistance-Effort', 'Correct contact', 'Quality of the relationship', 'Explanation of work to be done', 'Explanation of work carried out', 'Authorisation before additional work', 'Honesty-Confidence', 'Explanation of invoice', 'Clarity-transmission of information', 'Information regarding the progress of the work', 'Wait for appointment', 'Respect timeframe for work', 'Time taken for work', 'Availability of parts', 'Wait in reception', 'Time dedicated to me', 'Efficiency of the organisation', 'Delivery time', 'Price', 'Value for money', 'Respect of price and promises', 'Refund-Goodwill gesture', 'Part-exchange', 'Doing 100% what is asked', 'Impression of competence', 'Quality of work carried out', 'Attention to detail', 'Problem not diagnosed-not resolved', 'Return of vehicle', 'Cleanliness-State of vehicle', 'Quality of

## Loading and training Mistral 7B model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os, torch, platform, warnings
from trl import SFTTrainer

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

In [ ]:
# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

(True, True)

Let's example how well the model does at this task currently:

In [ ]:
def generate_response(prompt):
    encoded_input = tokenizer(prompt, return_tensors = "pt", add_special_tokens = True)
    model_inputs = encoded_input.to('cuda')

    generated_ids = model.generate(**model_inputs, max_new_tokens = 1000, do_sample = True, pad_token_id = tokenizer.eos_token_id)

    decoded_output = tokenizer.batch_decode(generated_ids)

    return decoded_output[0]

In [ ]:
generate_response("<s>[INST]### Instruction:\n Classify this text in one or more of the following categories:\n ['Welcome-Kindness-Warmth-Friendliness', 'Listenning-Care', 'Attention-Assistance-Effort', 'Correct contact', 'Quality of the relationship', 'Explanation of work to be done', 'Explanation of work carried out', 'Authorisation before additional work', 'Honesty-Confidence', 'Explanation of invoice', 'Clarity-transmission of information', 'Information regarding the progress of the work', 'Wait for appointment', 'Respect timeframe for work', 'Time taken for work', 'Availability of parts', 'Wait in reception', 'Time dedicated to me', 'Efficiency of the organisation', 'Delivery time', 'Price', 'Value for money', 'Respect of price and promises', 'Refund-Goodwill gesture', 'Part-exchange', 'Doing 100% what is asked', 'Impression of competence', 'Quality of work carried out', 'Attention to detail', 'Problem not diagnosed-not resolved', 'Return of vehicle', 'Cleanliness-State of vehicle', 'Quality of documents provided', 'Registration', 'Conformity of delivery', 'Availability of desired vehicle', 'Test drive', 'Condition of vehicle on delivery', 'Mobility-Courtesy car', 'Finance', 'Service contract', 'Connected services', 'Warranty', 'My Renault-My Dacia', 'Ease of parking', 'Ease of access-Proximity', 'Opening times', 'Reachability', 'Assistance-Breakdown cover', 'Ease to book appointment', 'Contact after sale', 'Contact after repairs', 'Request to be contacted', 'Manufacturing fault-Breakdown', 'Vehicle performance', 'Accessories', 'Quality of delivery', 'Vehicle handover', 'Comparison with competitors', 'Recommendation-intended loyalty', 'General satisfaction', 'Client fidelity', 'Brand image', 'Loss of customer (ALERTE)', 'Legal risk (ALERTE)', 'Appearance of premises', 'Waiting area-Comfort', 'Showroom', 'Questionnaire comments', 'Pressure for evaluation (ALERTE)', 'No opinion', 'Verbatim not exploitable']\n [/INST]\n\n [INST]### Additional Instruction:\n For each of these categories, indicate whether the text is positive, neutral, or negative.\n [/INST]\n\n ### Input:\nGood Co-ordination and scheduling. Limited manpower for bodyshop work took unnecessary wait time.\n\n ### Response:\n[{'category': 'Wait for appointment', 'polarity': 'positive'}, {'category': 'Time taken for work', 'polarity': 'negative'}]</s>")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


'<s> [INST]### Instruction:\nUse the provided input to generate a response that identifies one or more categories and indicates the polarity (Positive, Negative, or Neutral) for each category in the text.[/INST]\n\n### Input:\n Always friendly and efficient. Car comes back nice and clean.\n\n### Response:</s>\n Category: Customer Service\n Polarity: Positive\n\n Category: Cleanliness\n Polarity: Positive</s>'

In [ ]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
    )
model = get_peft_model(model, peft_config)

All that's left to do is set up a number of hyperparameters.

In [ ]:
OUTPUT_DIR = "mistral_instruct_generation"
%load_ext tensorboard
%tensorboard --logdir mistral_instruct_generation/runs

In [ ]:
# Training Arguments
# Hyperparameters should be adjusted based on the hardware you using
training_arguments = TrainingArguments(
    output_dir= OUTPUT_DIR,
    report_to = "tensorboard",
    num_train_epochs= 1,
    per_device_train_batch_size= 4,
    gradient_accumulation_steps= 2,
    optim = "paged_adamw_8bit",
    save_steps= 5000,
    logging_steps= 30,
    learning_rate= 2e-4,
    weight_decay= 0.001,
    fp16= False,
    bf16= False,
    max_grad_norm= 0.3,
    max_steps= -1,
    warmup_ratio= 0.3,
    group_by_length= True,
    lr_scheduler_type= "constant",
)
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    formatting_func = formatting_prompts_func,
    train_dataset=dataset['train'],
    eval_dataset = dataset['test'],
    peft_config=peft_config,
    max_seq_length= 2048,
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:282: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


Train the dataset on Mistral model:

In [ ]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
30,1.290200
60,0.964400
90,0.747300
120,0.851800
150,0.661700
180,0.885300
210,0.801100
240,0.712800
270,0.832600
300,0.667500


TrainOutput(global_step=375, training_loss=0.8257147394816081, metrics={'train_runtime': 2449.0034, 'train_samples_per_second': 1.225, 'train_steps_per_second': 0.153, 'total_flos': 1.967265528859853e+16, 'train_loss': 0.8257147394816081, 'epoch': 1.0})

## Evaluation process

In [ ]:
trainer.save_model("mistral_instruct_generation")

In [ ]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0]

In [ ]:
generate_response("<s>[INST]### Instruction:\n Classify this text in one or more of the following categories:\n ['Welcome-Kindness-Warmth-Friendliness', 'Listenning-Care', 'Attention-Assistance-Effort', 'Correct contact', 'Quality of the relationship', 'Explanation of work to be done', 'Explanation of work carried out', 'Authorisation before additional work', 'Honesty-Confidence', 'Explanation of invoice', 'Clarity-transmission of information', 'Information regarding the progress of the work', 'Wait for appointment', 'Respect timeframe for work', 'Time taken for work', 'Availability of parts', 'Wait in reception', 'Time dedicated to me', 'Efficiency of the organisation', 'Delivery time', 'Price', 'Value for money', 'Respect of price and promises', 'Refund-Goodwill gesture', 'Part-exchange', 'Doing 100% what is asked', 'Impression of competence', 'Quality of work carried out', 'Attention to detail', 'Problem not diagnosed-not resolved', 'Return of vehicle', 'Cleanliness-State of vehicle', 'Quality of documents provided', 'Registration', 'Conformity of delivery', 'Availability of desired vehicle', 'Test drive', 'Condition of vehicle on delivery', 'Mobility-Courtesy car', 'Finance', 'Service contract', 'Connected services', 'Warranty', 'My Renault-My Dacia', 'Ease of parking', 'Ease of access-Proximity', 'Opening times', 'Reachability', 'Assistance-Breakdown cover', 'Ease to book appointment', 'Contact after sale', 'Contact after repairs', 'Request to be contacted', 'Manufacturing fault-Breakdown', 'Vehicle performance', 'Accessories', 'Quality of delivery', 'Vehicle handover', 'Comparison with competitors', 'Recommendation-intended loyalty', 'General satisfaction', 'Client fidelity', 'Brand image', 'Loss of customer (ALERTE)', 'Legal risk (ALERTE)', 'Appearance of premises', 'Waiting area-Comfort', 'Showroom', 'Questionnaire comments', 'Pressure for evaluation (ALERTE)', 'No opinion', 'Verbatim not exploitable']\n [/INST]\n\n [INST]### Additional Instruction:\n For each of these categories, indicate whether the text is positive, neutral, or negative.\n [/INST]\n\n ### Input:\nEverything was good from picking my vehicle and returning my vehicle with good service of the vechile\n\n ### Response:\n[{'category': 'Return of vehicle', 'polarity': 'positive'}]</s>", merged_model)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


'<s> [INST]### Instruction:\nUse the provided input to generate a response that identifies one or more categories and indicates the polarity (Positive, Negative, or Neutral) for each category in the text.[/INST]\n\n### Input:\n Always friendly and efficient. Car comes back nice and clean.\n\n### Response:</s><s> QUIALITY OF SERVICE:\n### Positive</s>'

In [ ]:
generate_response("<s>[INST]### Instruction:\n Classify this text in one or more of the following categories:\n ['Welcome-Kindness-Warmth-Friendliness', 'Listenning-Care', 'Attention-Assistance-Effort', 'Correct contact', 'Quality of the relationship', 'Explanation of work to be done', 'Explanation of work carried out', 'Authorisation before additional work', 'Honesty-Confidence', 'Explanation of invoice', 'Clarity-transmission of information', 'Information regarding the progress of the work', 'Wait for appointment', 'Respect timeframe for work', 'Time taken for work', 'Availability of parts', 'Wait in reception', 'Time dedicated to me', 'Efficiency of the organisation', 'Delivery time', 'Price', 'Value for money', 'Respect of price and promises', 'Refund-Goodwill gesture', 'Part-exchange', 'Doing 100% what is asked', 'Impression of competence', 'Quality of work carried out', 'Attention to detail', 'Problem not diagnosed-not resolved', 'Return of vehicle', 'Cleanliness-State of vehicle', 'Quality of documents provided', 'Registration', 'Conformity of delivery', 'Availability of desired vehicle', 'Test drive', 'Condition of vehicle on delivery', 'Mobility-Courtesy car', 'Finance', 'Service contract', 'Connected services', 'Warranty', 'My Renault-My Dacia', 'Ease of parking', 'Ease of access-Proximity', 'Opening times', 'Reachability', 'Assistance-Breakdown cover', 'Ease to book appointment', 'Contact after sale', 'Contact after repairs', 'Request to be contacted', 'Manufacturing fault-Breakdown', 'Vehicle performance', 'Accessories', 'Quality of delivery', 'Vehicle handover', 'Comparison with competitors', 'Recommendation-intended loyalty', 'General satisfaction', 'Client fidelity', 'Brand image', 'Loss of customer (ALERTE)', 'Legal risk (ALERTE)', 'Appearance of premises', 'Waiting area-Comfort', 'Showroom', 'Questionnaire comments', 'Pressure for evaluation (ALERTE)', 'No opinion', 'Verbatim not exploitable']\n [/INST]\n\n [INST]### Additional Instruction:\n For each of these categories, indicate whether the text is positive, neutral, or negative.\n [/INST]\n\n ### Input:\nGood experience with Renault service and great meeting with manger\n\n ### Response:\n[{'category': 'Welcome-Kindness-Warmth-Friendliness', 'polarity': 'positive'}]</s>", merged_model)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


"<s> [INST]### Instruction:\nUse the provided input to generate a response that identifies one or more categories and indicates the polarity (Positive, Negative, or Neutral) for each category in the text.[/INST]\n\n### Input:\n Service incharge Mujjafar is knowledgeable and understands the problem and provides good service\n\n### Response:</s> [{'category': 'Service incharge competency', 'polarity': 'Positive'}]</s>"

In [ ]:
generate_response("<s>[INST]### Instruction:\n Classify this text in one or more of the following categories:\n ['Welcome-Kindness-Warmth-Friendliness', 'Listenning-Care', 'Attention-Assistance-Effort', 'Correct contact', 'Quality of the relationship', 'Explanation of work to be done', 'Explanation of work carried out', 'Authorisation before additional work', 'Honesty-Confidence', 'Explanation of invoice', 'Clarity-transmission of information', 'Information regarding the progress of the work', 'Wait for appointment', 'Respect timeframe for work', 'Time taken for work', 'Availability of parts', 'Wait in reception', 'Time dedicated to me', 'Efficiency of the organisation', 'Delivery time', 'Price', 'Value for money', 'Respect of price and promises', 'Refund-Goodwill gesture', 'Part-exchange', 'Doing 100% what is asked', 'Impression of competence', 'Quality of work carried out', 'Attention to detail', 'Problem not diagnosed-not resolved', 'Return of vehicle', 'Cleanliness-State of vehicle', 'Quality of documents provided', 'Registration', 'Conformity of delivery', 'Availability of desired vehicle', 'Test drive', 'Condition of vehicle on delivery', 'Mobility-Courtesy car', 'Finance', 'Service contract', 'Connected services', 'Warranty', 'My Renault-My Dacia', 'Ease of parking', 'Ease of access-Proximity', 'Opening times', 'Reachability', 'Assistance-Breakdown cover', 'Ease to book appointment', 'Contact after sale', 'Contact after repairs', 'Request to be contacted', 'Manufacturing fault-Breakdown', 'Vehicle performance', 'Accessories', 'Quality of delivery', 'Vehicle handover', 'Comparison with competitors', 'Recommendation-intended loyalty', 'General satisfaction', 'Client fidelity', 'Brand image', 'Loss of customer (ALERTE)', 'Legal risk (ALERTE)', 'Appearance of premises', 'Waiting area-Comfort', 'Showroom', 'Questionnaire comments', 'Pressure for evaluation (ALERTE)', 'No opinion', 'Verbatim not exploitable']\n [/INST]\n\n [INST]### Additional Instruction:\n For each of these categories, indicate whether the text is positive, neutral, or negative.\n [/INST]\n\n ### Input:\nDoing Well - Very attendive initial approach.\nTo be Improved - Followup/update to the customer can be in the same way.\n\n ### Response:\n[{'category': 'Listenning-Care', 'polarity': 'positive'}, {'category': 'Followup/update to the customer', 'polarity': 'negative'}]</s>", merged_model)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


"<s> [INST]### Instruction:\nUse the provided input to generate a response that identifies one or more categories and indicates the polarity (Positive, Negative, or Neutral) for each category in the text.[/INST]\n\n### Input:\n THERE WAS AN EML LIGHT ON WHICH HAS NOT BEEN CHECKED.  AND YOU HAVEN'T TOLD ME THE REASON OF IT. IM NOT HAPPY OF YOUR SERVICE\n\n### Response:</s><s> Questionnaire is a bit out of context. EML light was not checked at all in service workshop.\n\n### Category:\nIssue not addressed\n\n### Polarity:\nNegative</s>"